<a href="https://colab.research.google.com/github/YasmineSarraj/M3-Assignments-Deep-Learning/blob/main/sBERT_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# sBERT

In [3]:
!pip install sentence_transformers -q

In [4]:
from sentence_transformers import SentenceTransformer, util
from PIL import Image

In [5]:
#Load CLIP model
model = SentenceTransformer('clip-ViT-B-32')


# Request images

In [8]:
import requests
import os
import numpy as np
import heapq

In [33]:
urls = ["https://img01.ztat.net/article/spp-media-p1/8f00a0bb65e639aba317b8348f4e5133/bd18006cfce44e55ae0c79aca0ffa126.jpg?imwidth=1800",
        "https://img01.ztat.net/article/spp-media-p1/1c4b7f5c261242da858e87b116954b78/f7ab75e48be84a038898a16b386ff643.jpg?imwidth=1800",
        "https://img01.ztat.net/article/spp-media-p1/4a7c9f4d49ec4fefa4fa98f02e4fcf07/cb65ec624de84ba38f7047abff88cac7.jpg?imwidth=1800",
        "https://img01.ztat.net/article/spp-media-p1/05f45c6f11f149afa6fa7f1cb320b723/73b37d29c5744b67b4b24c44e8d55ede.jpg?imwidth=1800",
        "https://img01.ztat.net/article/spp-media-p1/7deb74ac32e9431f887e6404065b7f34/eed4091560904f25bdce000abb9fe024.jpg?imwidth=1800",
        "https://img01.ztat.net/article/spp-media-p1/0a0cfb1e3fa54d4a8b58982ed5117ff7/f941f093f2364b8bab53737219241d8b.jpg?imwidth=1800",
        "https://img01.ztat.net/article/spp-media-p1/7152cd40305943bfb6dce4eeb1fd598a/0c6cb7f6d4214017a077732d52239274.jpg?imwidth=1800"]

In [45]:
dress_urls = ["https://en.zalando.de/levisr-selma-dress-denim-dress-going-steady-le221c01r-k11.html",
              "https://en.zalando.de/buffalo-maxi-dress-blau-bedruckt-bu321c01f-k11.html",
              "https://en.zalando.de/nike-sportswear-midi-dress-shift-dress-whiteblack-ni121c04e-a11.html",
              "https://en.zalando.de/adidas-originals-next-dress-day-dress-black-ad121c0ch-q11.html",
              "https://en.zalando.de/champion-jersey-dress-pink-c7641l002-j11.html",
              "https://en.zalando.de/buffalo-jersey-dress-schwarz-rosa-bedruckt-bu321c00p-q11.html",
              "https://en.zalando.de/adidas-originals-shirt-dress-off-white-ad121c079-a11.html"]

In [35]:
print(len(urls))
print(len(dress_urls))

7
7


In [36]:
for idx, url in enumerate(urls):
  with open(f"/content/imgs/dress_{idx}.jpg", "wb") as f:
    img_data = requests.get(url).content
    f.write(img_data)

# Encode images

In [37]:
imgs_enc = []
img_paths = os.listdir("/content/imgs")

for img_path in img_paths:
  img_emb = model.encode(Image.open(f"/content/imgs/{img_path}"))
  imgs_enc.append(img_emb)

In [39]:
prompt = "summer dress"

In [40]:
cos_sims = util.cos_sim(model.encode(prompt), imgs_enc)[0]
cos_sims

tensor([0.2083, 0.2269, 0.2489, 0.2391, 0.2555, 0.2793, 0.2439])

In [41]:
heapq.nlargest(3, range(len(cos_sims)), cos_sims.__getitem__)

[5, 4, 2]

# Grad.io interface

In [42]:
import gradio as gr

In [43]:
  # best_fits = heapq.nlargest(3, range(len(cos_sims)), cos_sims.__getitem__)
  # best_fit = np.argmax(cos_sims).item()


In [46]:
def show_best_img(prompt):
  cos_sims = util.cos_sim(model.encode(prompt), imgs_enc)[0]
  best_fits = heapq.nlargest(3, range(len(cos_sims)), cos_sims.__getitem__)
  return f"/content/imgs/dress_{best_fits[0]}.jpg", dress_urls[best_fits[0]], f"/content/imgs/dress_{best_fits[1]}.jpg", dress_urls[best_fits[1]], f"/content/imgs/dress_{best_fits[2]}.jpg", dress_urls[best_fits[2]]

demo = gr.Interface(
    fn=show_best_img,
    inputs=gr.Textbox(lines=1, placeholder="Describe the dress you are looking for ..."),
    outputs=["image", "text", "image", "text", "image", "text"]
)
demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>